## Proyecto | Predicción de Delitos en la colonia Roma Norte

In [229]:
#importar librerías
import pandas as pd
import numpy as np

import os

import folium
from folium import plugins

from wwo_hist import retrieve_hist_data

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import normalize
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_csv('../Data/carpetas-de-investigacion-pgj-de-la-ciudad-de-mexico.csv', sep=';')

In [3]:
data.head(10)

,ao_hechos,mes_hechos,fecha_hechos,delito,categoria_delito,fiscalia,agencia,unidad_investigacion,alcaldia_hechos,colonia_hechos,ao_inicio,mes_inicio,fecha_inicio,calle_hechos,calle_hechos2,longitud,latitud,geopoint
0,2016.0,Diciembre,2016-12-31 22:11:00,HOMICIDIO CULPOSO POR TRÁNSITO VEHICULAR (ATRO...,DELITO DE BAJO IMPACTO,INVESTIGACIÓN EN TLAHUAC,TLH-1,UI-2SD,TLAHUAC,SANTA ANA PONIENTE,2017,Enero,2017-01-01 00:37:27,AVENIDA TLAHUAC,TURANDOT,-99.053535,19.302084,"19.3020842299,-99.0535354187"
1,2016.0,Diciembre,2016-12-31 19:55:00,ROBO DE VEHICULO DE SERVICIO PARTICULAR CON VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,INVESTIGACIÓN EN IZTAPALAPA,IZP-4,UI-2SD,IZTAPALAPA,UNIDAD MODELO,2017,Enero,2017-01-01 01:29:37,RIO CHURUBUSCO,NaN,-99.128550,19.363062,"19.3630615318,-99.1285499097"
2,2017.0,Enero,2017-01-01 01:15:00,LESIONES INTENCIONALES POR ARMA BLANCA,DELITO DE BAJO IMPACTO,INVESTIGACIÓN EN CUAUHTEMOC,CUH-2,UI-2CD,CUAUHTEMOC,CENTRO,2017,Enero,2017-01-01 02:01:49,AVENIDA ANILLO DE CIRCUNVALACION,FRAY SERVANDO TERESA DE MIER,-99.125428,19.427148,"19.4271484491,-99.1254284961"
3,2017.0,Enero,2017-01-01 00:00:00,LESIONES CULPOSAS,DELITO DE BAJO IMPACTO,INVESTIGACIÓN EN CUAUHTEMOC,CUH-2,UI-2CD,CUAUHTEMOC,DOCTORES,2017,Enero,2017-01-01 02:55:04,AVENIDA EJE CENTRAL Y DOCTOR MARQUEZ,NaN,-99.144286,19.410851,"19.410850532,-99.1442862319"
4,2011.0,Agosto,2011-08-17 12:00:00,FRAUDE,DELITO DE BAJO IMPACTO,INVESTIGACIÓN EN CUAUHTEMOC,CUH-8,UI-2SD,CUAUHTEMOC,DOCTORES,2017,Enero,2017-01-01 03:23:31,AV. NIÑOS HEROES,NaN,-99.150120,19.421071,"19.4210705569,-99.1501195725"
5,2015.0,Octubre,2015-10-07 12:00:00,FRAUDE,DELITO DE BAJO IMPACTO,INVESTIGACIÓN EN CUAUHTEMOC,CUH-8,UI-2SD,CUAUHTEMOC,DOCTORES,2017,Enero,2017-01-01 03:58:27,AV. NIÑOS HEROES,NaN,-99.150120,19.421071,"19.4210705569,-99.1501195725"
6,2017.0,Enero,2017-01-01 03:23:00,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,INVESTIGACIÓN EN VENUSTIANO CARRANZA,VC-3,UI-2CD,VENUSTIANO CARRANZA,PUEBLA,2017,Enero,2017-01-01 04:16:12,CALZADA DE IGNACIO ZARAGOZA NUMERO 612,NaN,-99.083968,19.413197,"19.4131971924,-99.0839684829"
7,2017.0,Enero,2017-01-01 02:10:00,DAÑO EN PROPIEDAD AJENA CULPOSA,DELITO DE BAJO IMPACTO,INVESTIGACIÓN EN CUAUHTEMOC,CUH-8,UI-2SD,CUAUHTEMOC,OBRERA,2017,Enero,2017-01-01 04:55:05,MANUEL GUTIEREZ NAJERA,NaN,-99.136163,19.415324,"19.4153238588,-99.1361634753"
8,2017.0,Enero,2017-01-01 02:29:00,LESIONES INTENCIONALES POR ARMA BLANCA,DELITO DE BAJO IMPACTO,INVESTIGACIÓN EN CUAUHTEMOC,CUH-4,UI-2SD,CUAUHTEMOC,NaN,2017,Enero,2017-01-01 06:47:21,NOTIF. HOSPITAL DR. GREGORIO SALAS FLORES,- NO SCINCE -,NaN,NaN,NaN
9,2017.0,Enero,2017-01-01 07:00:00,HOMICIDIO CULPOSO POR ARMA DE FUEGO,DELITO DE BAJO IMPACTO,INVESTIGACIÓN EN GUSTAVO A. MADERO,GAM-6,UI-2CD,GUSTAVO A MADERO,GRANJAS MODERNAS,2017,Enero,2017-01-01 06:51:59,AV. SAN JUAN DE ARAGON,CALLE PELICANOS,-99.103234,19.476753,"19.476752974,-99.1032341417"


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829679 entries, 0 to 829678
Data columns (total 18 columns):
ao_hechos               829480 non-null float64
mes_hechos              829480 non-null object
fecha_hechos            829497 non-null object
delito                  829679 non-null object
categoria_delito        829679 non-null object
fiscalia                829679 non-null object
agencia                 829679 non-null object
unidad_investigacion    829454 non-null object
alcaldia_hechos         826563 non-null object
colonia_hechos          797058 non-null object
ao_inicio               829679 non-null int64
mes_inicio              829679 non-null object
fecha_inicio            829679 non-null object
calle_hechos            827341 non-null object
calle_hechos2           335837 non-null object
longitud                797811 non-null float64
latitud                 797811 non-null float64
geopoint                797811 non-null object
dtypes: float64(3), int64(1), object(14)


In [5]:
data.categoria_delito.value_counts()

DELITO DE BAJO IMPACTO                                     631312
ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA        49216
HECHO NO DELICTIVO                                          49090
ROBO DE VEHÍCULO CON Y SIN VIOLENCIA                        46306
ROBO A NEGOCIO CON VIOLENCIA                                13599
ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VIOLENCIA        9348
ROBO A REPARTIDOR CON Y SIN VIOLENCIA                        7847
LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO                6988
HOMICIDIO DOLOSO                                             4783
VIOLACIÓN                                                    3250
ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN VIOLENCIA      3091
ROBO A CASA HABITACIÓN CON VIOLENCIA                         1938
ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON VIOLENCIA      1516
ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA                 789
ROBO A TRANSPORTISTA CON Y SIN VIOLENCIA                      472
SECUESTRO 

In [107]:
data.delito.value_counts()

VIOLENCIA FAMILIAR                                                      70106
ROBO A NEGOCIO SIN VIOLENCIA                                            53623
ROBO DE OBJETOS                                                         52345
FRAUDE                                                                  45601
ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA                          43330
AMENAZAS                                                                38040
DENUNCIA DE HECHOS                                                      36366
ROBO DE ACCESORIOS DE AUTO                                              24903
ROBO DE OBJETOS DEL INTERIOR DE UN VEHICULO                             23579
ROBO A CASA HABITACION SIN VIOLENCIA                                    22374
FALSIFICACION DE TITULOS AL PORTADOR Y DOCUMENTOS DE CREDITO PUBLICO    22063
ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VIOLENCIA                   21707
ROBO A TRANSEUNTE DE CELULAR CON VIOLENCIA                      

In [1049]:
data[data.categoria_delito == 'DELITO DE BAJO IMPACTO'].delito.value_counts()

VIOLENCIA FAMILIAR                                                      70106
ROBO A NEGOCIO SIN VIOLENCIA                                            53623
ROBO DE OBJETOS                                                         52345
FRAUDE                                                                  45601
AMENAZAS                                                                38040
ROBO DE ACCESORIOS DE AUTO                                              24903
ROBO DE OBJETOS DEL INTERIOR DE UN VEHICULO                             23579
ROBO A CASA HABITACION SIN VIOLENCIA                                    22374
FALSIFICACION DE TITULOS AL PORTADOR Y DOCUMENTOS DE CREDITO PUBLICO    22063
ROBO A TRANSEUNTE DE CELULAR CON VIOLENCIA                              13981
ABUSO DE CONFIANZA                                                      13131
DAÑO EN PROPIEDAD AJENA CULPOSA POR TRÁNSITO VEHICULAR A AUTOMOVIL      12794
DESPOJO                                                         

In [109]:
data[data.categoria_delito == 'ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA'].delito.value_counts()

ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA             43330
ROBO A TRANSEUNTE EN VIA PUBLICA SIN VIOLENCIA              5332
ROBO A TRANSEUNTE EN VIA PUBLICA (NOMINA) CON VIOLENCIA       19
Name: delito, dtype: int64

In [6]:
data.drop(columns=['ao_hechos', 'mes_hechos', 'calle_hechos2', 'geopoint', 'ao_inicio', 'mes_inicio', 'fecha_inicio'], inplace=True)

In [7]:
data['fecha_hechos'] = pd.to_datetime(data.fecha_hechos, errors='coerce')

In [8]:
data.longitud.astype('float', inplace=True)
data.latitud.astype('float', inplace=True)

0         19.302084
1         19.363062
2         19.427148
3         19.410851
4         19.421071
5         19.421071
6         19.413197
7         19.415324
8               NaN
9         19.476753
10        19.485339
11        19.472496
12        19.346451
13        19.337518
14        19.350219
15        19.432277
16        19.291589
17        19.421663
18        19.306929
19        19.470465
20        19.350121
21        19.459908
22        19.480559
23        19.306275
24        19.298845
25        19.270806
26        19.438991
27        19.318823
28        19.433659
29        19.317426
            ...    
829649    19.395197
829650    19.325530
829651    19.270393
829652    19.374895
829653    19.315210
829654    19.479527
829655    19.395032
829656    19.457644
829657    19.438072
829658    19.287737
829659    19.498635
829660    19.352901
829661    19.437556
829662    19.217099
829663    19.394783
829664    19.525792
829665    19.325653
829666    19.493782
829667    19.388654


In [9]:
data.dropna(inplace=True)

In [541]:
roma_n = data[(data.alcaldia_hechos == 'CUAUHTEMOC') & (data.colonia_hechos == 'ROMA NORTE') & (data.categoria_delito == 'ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA')]

In [459]:
roma_n.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 657 entries, 3246 to 826883
Data columns (total 11 columns):
fecha_hechos            657 non-null datetime64[ns]
delito                  657 non-null object
categoria_delito        657 non-null object
fiscalia                657 non-null object
agencia                 657 non-null object
unidad_investigacion    657 non-null object
alcaldia_hechos         657 non-null object
colonia_hechos          657 non-null object
calle_hechos            657 non-null object
longitud                657 non-null float64
latitud                 657 non-null float64
dtypes: datetime64[ns](1), float64(2), object(8)
memory usage: 61.6+ KB


In [589]:
roma_n.sort_values(by='fecha_hechos', ascending=True, inplace=True)

In [543]:
roma_n = pd.merge_asof(roma_n, clima, on='fecha_hechos')

In [544]:
roma_n['nombre_dia'] = roma_n.fecha_hechos.dt.weekday

In [1004]:
festivos = ['01-01', '01-05', '01-06', '02-05', '02-04', '03-21', '05-01', '05-05', '09-15', '09-16', '10-31', '11-01', '11-02', '11-20', '12-12', '12-24', '12-25', '12-31']
años = ['2015', '2016', '2017', '2018', '2019']
festivo = []
for i in años:
    for x in festivos:
        festivo.append(i+'-'+x)

In [1005]:
festivo = pd.DataFrame(festivo, columns=['dia'])
festivo['dia_festivo'] = 1
festivo['dia'] = pd.to_datetime(festivo.dia).dt.date

In [963]:
festivo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 2 columns):
dia            90 non-null object
dia_festivo    90 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.5+ KB


In [1039]:
prueba3 = roma_n.copy()

In [1040]:
prueba3['dia'] = prueba3.fecha_hechos.dt.date

In [1041]:
prueba3 = pd.merge(prueba3, festivo, how='left', on='dia')

In [1046]:
prueba3.dia_festivo.fillna(0, inplace=True)
prueba3['dia_festivo'] = prueba3.dia_festivo.astype('int', inplace=True)

In [1047]:
prueba3.dia_festivo.value_counts()

0    629
1     28
Name: dia_festivo, dtype: int64

In [1048]:
prueba3

,fecha_hechos,delito,categoria_delito,fiscalia,agencia,unidad_investigacion,alcaldia_hechos,colonia_hechos,calle_hechos,longitud,...,humedad,precipitacion,presion,temperatura,visibilidad,dir_viento,vel_viento,nombre_dia,dia,dia_festivo
0,2015-04-17 02:00:00,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,INVESTIGACIÓN EN BENITO JUÁREZ,BJ-1,UI-2CD,CUAUHTEMOC,ROMA NORTE,NUEVOLEON (AV. OAXACA),-99.168124,...,40,0.0,1015,16,10,194,7,4,2015-04-17,0
1,2015-11-30 15:00:00,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,INVESTIGACIÓN EN CUAUHTEMOC,CUH-7,UI-1SD,CUAUHTEMOC,ROMA NORTE,DURANGO,-99.166743,...,23,0.0,1011,23,10,259,4,0,2015-11-30,0
2,2015-12-09 01:40:00,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,INVESTIGACIÓN EN CUAUHTEMOC,CUH-7,UI-3SD,CUAUHTEMOC,ROMA NORTE,SONORA 57,-99.172206,...,73,0.0,1018,11,10,65,2,2,2015-12-09,0
3,2015-12-11 12:00:00,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,INVESTIGACIÓN EN CUAUHTEMOC,CUH-7,UI-3SD,CUAUHTEMOC,ROMA NORTE,COLIMA,-99.172064,...,39,0.0,1015,19,10,183,11,4,2015-12-11,0
4,2015-12-12 13:00:00,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,INVESTIGACIÓN EN CUAUHTEMOC,CUH-7,UI-3SD,CUAUHTEMOC,ROMA NORTE,CORDOBA,-99.158353,...,43,0.0,1013,20,10,177,14,5,2015-12-12,1
5,2016-01-05 15:20:00,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,INVESTIGACIÓN EN CUAUHTEMOC,CUH-2,UI-1CD,CUAUHTEMOC,ROMA NORTE,ORIZABA,-99.158182,...,32,0.0,1015,18,10,239,15,1,2016-01-05,1
6,2016-01-05 20:00:00,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,INVESTIGACIÓN EN CUAUHTEMOC,CUH-7,UI-2CD,CUAUHTEMOC,ROMA NORTE,AVENIDA CUAUHTEMOC,-99.154881,...,53,0.0,1017,13,10,196,11,1,2016-01-05,1
7,2016-01-09 03:30:00,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,INVESTIGACIÓN EN CUAUHTEMOC,CUH-7,UI-2CD,CUAUHTEMOC,ROMA NORTE,MEDELLIN,-99.165555,...,38,0.0,1015,9,10,192,9,5,2016-01-09,0
8,2016-01-16 03:30:00,ROBO A TRANSEUNTE EN VIA PUBLICA SIN VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,INVESTIGACIÓN EN CUAUHTEMOC,CUH-7,UI-1SD,CUAUHTEMOC,ROMA NORTE,AV. MONTERREY,-99.164816,...,29,0.0,1015,8,10,193,8,5,2016-01-16,0
9,2016-01-20 02:40:00,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,INVESTIGACIÓN EN CUAUHTEMOC,CUH-2,UI-3CD,CUAUHTEMOC,ROMA NORTE,ORIZABA,-99.160791,...,64,0.0,1021,10,10,29,3,2,2016-01-20,0


In [470]:
mapa=folium.Map(location=[19.41641111, -99.16169722], zoom_start=15)

In [148]:
for index, row in roma_n.iterrows():
    folium.CircleMarker([row['latitud'], row['longitud']],
                        radius=1,
                        #popup=df['delito'],
                        fill_color="#3db7e4",
                       ).add_to(mapa)

In [149]:
geo = roma_n[['latitud', 'longitud']].as_matrix()

mapa.add_children(plugins.HeatMap(geo, radius=15))

C:\Users\Eduardo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\Users\Eduardo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [150]:
mapa.save('mapa.html')

In [396]:
prueba = roma_n.copy()

In [397]:
prueba.drop(columns=['fiscalia', 'agencia', 'unidad_investigacion', 'alcaldia_hechos', 'colonia_hechos'], inplace=True)

In [398]:
prueba.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 657 entries, 0 to 656
Data columns (total 19 columns):
fecha_hechos        657 non-null datetime64[ns]
delito              657 non-null object
categoria_delito    657 non-null object
calle_hechos        657 non-null object
longitud            657 non-null float64
latitud             657 non-null float64
uv                  657 non-null int64
ilu_luna            657 non-null int64
punto_rocio         657 non-null int64
temp_sentir         657 non-null int64
nubosidad           657 non-null int64
humedad             657 non-null int64
precipitacion       657 non-null float64
presion             657 non-null int64
temperatura         657 non-null int64
visibilidad         657 non-null int64
dir_viento          657 non-null int64
vel_viento          657 non-null int64
nombre_dia          657 non-null int64
dtypes: datetime64[ns](1), float64(3), int64(12), object(3)
memory usage: 102.7+ KB


In [399]:
X = prueba.drop(columns=['fecha_hechos', 'latitud', 'longitud'])
y = prueba[['fecha_hechos', 'latitud', 'longitud']]

In [400]:
label = LabelEncoder()
X.delito = label.fit_transform(X.delito)
X.categoria_delito = label.fit_transform(X.categoria_delito)
X.calle_hechos = label.fit_transform(X.calle_hechos)
X['año'] = y.fecha_hechos.dt.year

In [401]:
process = StandardScaler()
X = process.fit_transform(X)

C:\Users\Eduardo\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Eduardo\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [403]:
X

array([[-0.39068092,  0.        ,  0.56435976, ...,  0.24410932,
         0.44353276, -3.00038115],
       [-0.39068092,  0.        , -0.34870945, ..., -0.58117278,
        -1.58008547, -3.00038115],
       [-0.39068092,  0.        ,  1.39916589, ..., -1.13136085,
        -0.56827635, -3.00038115],
       ...,
       [-0.39068092,  0.        , -0.00956946, ..., -1.13136085,
        -0.06237179,  1.54953326],
       [-0.39068092,  0.        ,  1.29481512, ...,  0.24410932,
         0.44353276,  1.54953326],
       [-0.39068092,  0.        ,  0.98176282, ..., -0.30607875,
         1.45534188,  1.54953326]])

In [404]:
y['dia'] = y.fecha_hechos.dt.day
y['mese'] = y.fecha_hechos.dt.month
y['hora'] = y.fecha_hechos.dt.hour
y['minuto'] = y.fecha_hechos.dt.minute

C:\Users\Eduardo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Eduardo\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Eduardo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://p

In [405]:
y.drop(columns='fecha_hechos', inplace=True)

C:\Users\Eduardo\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [406]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [417]:
model = RandomForestRegressor(n_estimators=500)
model.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [418]:
pre = model.predict(X)

In [457]:
r2_score(y, pre)

0.8813242202018353

In [451]:
prediction = pd.DataFrame(pre, columns=['latitud', 'longitud', 'day', 'month', 'hour', 'minute'])
prediction[['day', 'month', 'hour', 'minute']] = prediction[['day', 'month', 'hour', 'minute']].round().astype('int')
prediction['year'] = '2020'

In [452]:
prediction['fecha_hechos'] = pd.to_datetime(prediction[['year', 'day', 'month', 'hour', 'minute']], errors='coerce')

In [454]:
prediction.drop(columns=['year', 'day', 'month', 'hour', 'minute'], inplace=True)

In [455]:
prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 657 entries, 0 to 656
Data columns (total 3 columns):
latitud         657 non-null float64
longitud        657 non-null float64
fecha_hechos    657 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(2)
memory usage: 15.5 KB


In [456]:
prediction.head()

,latitud,longitud,fecha_hechos
0,19.418526,-99.166456,2020-05-17 05:07:00
1,19.419210,-99.165258,2020-09-25 15:07:00
2,19.418514,-99.169247,2020-11-12 04:32:00
3,19.417696,-99.169011,2020-10-12 13:06:00
4,19.417965,-99.160967,2020-11-13 14:05:00


In [1050]:
os.chdir("../Data")

In [1051]:
frequency=1
start_date = '01-DEC-2014'
end_date = '30-AUG-2019'
api_key = '28f7f02aa28d4afe9dc215223190509'
location_list = ['mexico_city']

hist_weather_data = retrieve_hist_data(api_key,
                                location_list,
                                start_date,
                                end_date,
                                frequency,
                                location_label = False,
                                export_csv = True,
                                store_df = True)



Retrieving weather data for mexico_city


Currently retrieving data for mexico_city: from 2014-12-01 to 2014-12-31
Time elapsed (hh:mm:ss.ms) 0:00:07.212004
Currently retrieving data for mexico_city: from 2015-01-01 to 2015-01-31
Time elapsed (hh:mm:ss.ms) 0:00:12.377554
Currently retrieving data for mexico_city: from 2015-02-01 to 2015-02-28
Time elapsed (hh:mm:ss.ms) 0:00:16.672364
Currently retrieving data for mexico_city: from 2015-03-01 to 2015-03-31
Time elapsed (hh:mm:ss.ms) 0:00:21.488435
Currently retrieving data for mexico_city: from 2015-04-01 to 2015-04-30
Time elapsed (hh:mm:ss.ms) 0:00:25.926191
Currently retrieving data for mexico_city: from 2015-05-01 to 2015-05-31
Time elapsed (hh:mm:ss.ms) 0:00:30.404104
Currently retrieving data for mexico_city: from 2015-06-01 to 2015-06-30
Time elapsed (hh:mm:ss.ms) 0:00:35.790787
Currently retrieving data for mexico_city: from 2015-07-01 to 2015-07-31
Time elapsed (hh:mm:ss.ms) 0:00:41.237322
Currently retrieving data for mexico

In [1052]:
clima = pd.read_csv('../Data/mexico_city.csv')

In [1053]:
clima['date_time'] = pd.to_datetime(clima.date_time, errors='coerce')

In [1054]:
clima.drop(columns=['maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour', 'uvIndex.1', 'moonrise', 'moonset',
                   'sunrise', 'sunset', 'HeatIndexC', 'WindChillC', 'WindGustKmph'], inplace=True)

In [1055]:
clima.columns = ['fecha_hechos', 'uv', 'ilu_luna', 'punto_rocio', 'temp_sentir', 'nubosidad', 'humedad', 
                      'precipitacion', 'presion', 'temperatura', 'visibilidad', 'dir_viento', 'vel_viento']

In [1056]:
clima.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41616 entries, 0 to 41615
Data columns (total 13 columns):
fecha_hechos     41616 non-null datetime64[ns]
uv               41616 non-null int64
ilu_luna         41616 non-null int64
punto_rocio      41616 non-null int64
temp_sentir      41616 non-null int64
nubosidad        41616 non-null int64
humedad          41616 non-null int64
precipitacion    41616 non-null float64
presion          41616 non-null int64
temperatura      41616 non-null int64
visibilidad      41616 non-null int64
dir_viento       41616 non-null int64
vel_viento       41616 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(11)
memory usage: 4.1 MB


In [1057]:
clima.head()

,fecha_hechos,uv,ilu_luna,punto_rocio,temp_sentir,nubosidad,humedad,precipitacion,presion,temperatura,visibilidad,dir_viento,vel_viento
0,2014-12-01 00:00:00,4,62,8,11,6,79,0.0,1019,9,10,28,5
1,2014-12-01 01:00:00,4,62,8,10,8,82,0.0,1019,9,10,21,6
2,2014-12-01 02:00:00,4,62,8,10,10,85,0.0,1019,9,10,14,6
3,2014-12-01 03:00:00,4,62,8,9,12,88,0.0,1019,8,10,7,6
4,2014-12-01 04:00:00,4,62,8,9,13,89,0.0,1019,8,10,104,5


In [364]:
prueba2 = prueba.copy()

In [365]:
prueba2.head()

,fecha_hechos,delito,categoria_delito,calle_hechos,longitud,latitud,nombre_dia
208538,2015-04-17 02:00:00,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,NUEVOLEON (AV. OAXACA),-99.168124,19.418106,4
251251,2015-11-30 15:00:00,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,DURANGO,-99.166743,19.419768,0
253583,2015-12-09 01:40:00,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,SONORA 57,-99.172206,19.418596,2
699939,2015-12-11 12:00:00,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,COLIMA,-99.172064,19.417163,4
218544,2015-12-12 13:00:00,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,CORDOBA,-99.158353,19.417733,5


In [370]:
final = pd.merge_asof(prueba2, clima, on='fecha_hechos')

In [383]:
final.tail()

,fecha_hechos,delito,categoria_delito,calle_hechos,longitud,latitud,nombre_dia,uv,ilu_luna,punto_rocio,temp_sentir,nubosidad,humedad,precipitacion,presion,temperatura,visibilidad,dir_viento,vel_viento
652,2019-07-06 11:30:00,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,3 PONIENTE (MEDELLÍN),-99.165501,19.415843,5,5,24,10,21,20,51,0.0,1019,21,10,15,10
653,2019-07-18 01:30:00,ROBO A TRANSEUNTE EN VIA PUBLICA SIN VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,AVENIDA INSURGENTES,-99.163485,19.421830,3,5,89,8,16,32,60,0.0,1020,16,10,148,3
654,2019-07-18 02:37:00,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,FRONTERA,-99.156013,19.419570,3,5,89,8,16,26,61,0.0,1019,16,10,115,2
655,2019-07-19 13:30:00,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,SAN LUIS POTOSI,-99.161249,19.413802,4,5,82,9,23,33,43,0.0,1016,23,10,55,7
656,2019-07-28 02:15:00,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,PUEBLA,-99.165054,19.422116,6,5,16,9,15,33,71,0.0,1021,15,10,20,5
